Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://manipulation.csail.mit.edu/intro.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- define some utility methods/classes that will eventually disappear from this notebook and live in drake.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# TODO(russt): install only drake (not manipulation) after colab gets to python3.8
if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='c1bdae733682f8a390f848bc6cb0dbbf9ea98602', drake_version='0.27.0', drake_build='release')

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

from manipulation import running_as_notebook

# TODO(russt): Consider teaching meshcat how to open the window on colab.
def open_window(url, name):
    if 'google.colab' in sys.modules:
        # This has the benefit of not persisting between notebooks if the output 
        # is accidentally saved.
        from google.colab import output
        output.eval_js(f'window.open("{url}", name);', ignore_result=True)
    else:
        from IPython.display import display, Javascript
        display(Javascript(f'window.open("{url}", name);'))

# Imports
import numpy as np
from ipywidgets import ToggleButton, ToggleButtons

import pydrake.all

from pydrake.all import RigidTransform, RotationMatrix
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from pydrake.examples.manipulation_station import ManipulationStation

pydrake.common.set_log_level("warn")

# Teleop Example (3D)

In this example, we assemble a diagram with all of the relevant subsystems (the manipulation station, the meshcat visualizer, and some systems that provide a minimal teleop interface and convert the teleop output from end-effector commands into joint commands.  We'll learn more about each of these components in the following chapters.

**NOTE:** If you command the robot to move its gripper beyond what is possible, then you get a message about "differential IK" failing.  I've left that in for now (rather than setting very conservative slider limits) partly because it has tutorial value.  We'll understand it more precisely soon!  For now, just stop the simulation and rerun the cell if you get stuck.


In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(ManipulationStation())

station.SetupClutterClearingStation()
#ycb_objects = CreateClutterClearingYcbObjectList()
#for model_file, X_WObject in ycb_objects:
#    station.AddManipulandFromFile(model_file, X_WObject)
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/"
    + "061_foam_brick.sdf",
    RigidTransform(RotationMatrix.Identity(), [0, -0.6, 0.2]))
station.Finalize()

visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
    builder, 
    station.get_scene_graph(), 
    station.GetOutputPort("query_object"),
    zmq_url="new",
    open_browser=False,
    server_args=server_args)
#    jupyter_comms=True)

robot = station.get_controller_plant()
params = pydrake.manipulation.planner.DifferentialInverseKinematicsParameters(
    robot.num_positions(), robot.num_velocities())

time_step = 0.005
params.set_timestep(time_step)
# True velocity limits for the IIWA14 (in rad, rounded down to the first
# decimal)
iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                  iiwa14_velocity_limits))
differential_ik = builder.AddSystem(
    pydrake.manipulation.planner.DifferentialInverseKinematicsIntegrator(
        robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
builder.Connect(differential_ik.get_output_port(),
                station.GetInputPort("iiwa_position"))

teleop = builder.AddSystem(PoseSliders(
    min_range = PoseSliders.MinRange(roll=0, pitch=-0.5, yaw=-np.pi, 
                                     x=-0.6, y=-0.8, z=0.0),
    max_range = PoseSliders.MaxRange(roll=2*np.pi, pitch=np.pi, yaw=np.pi,
                                     x=0.8, y=0.3, z=1.1)
))
builder.Connect(teleop.get_output_port(0), 
                differential_ik.get_input_port())
wsg_buttons = ToggleButtons(value=0.107, description="SchunkWsg", 
                            options=[('Open', 0.107), ('Close', 0.002)])
wsg_teleop = builder.AddSystem(WidgetSystem([wsg_buttons]))
builder.Connect(wsg_teleop.get_output_port(0),
                station.GetInputPort("wsg_position"))

diagram = builder.Build()
simulator = pydrake.systems.analysis.Simulator(diagram)
context = simulator.get_mutable_context()

station_context = station.GetMyMutableContextFromRoot(context)

q0 = station.GetOutputPort("iiwa_position_measured").Eval(
    station_context)
differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
differential_ik.SetPositions(diff_ik_context, q0)
teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

if running_as_notebook:  # Then we're not just running as a test on CI.
    simulator.set_target_realtime_rate(1.0)

    # Open the meshcat visualizer window (check your pop-up blocker).
    open_window(visualizer.vis.url(), "meshcat")

    stop_button = ToggleButton(value=False, description='Stop Simulation')
    display(stop_button)
    while not stop_button.value:
        simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
    stop_button.value = False
  
else:
    simulator.AdvanceTo(0.1)

# Teleop Example (2D)

Many of the core concepts in this class can be studied in 2D instead of 3D.  And everything is simpler/cleaner there,  including teleoperation!

In [ ]:
builder = pydrake.systems.framework.DiagramBuilder()

station = builder.AddSystem(ManipulationStation())
station.SetupPlanarIiwaStation()
station.AddManipulandFromFile(
    "drake/examples/manipulation_station/models/"
    + "061_foam_brick.sdf",
    RigidTransform(RotationMatrix.Identity(), [0.6, 0, 0]))
# TODO(russt): Add planar joint to brick
station.Finalize()

visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
    builder, 
    station.get_scene_graph(), 
    station.GetOutputPort("query_object"), 
    zmq_url="new",
    open_browser=False,
    server_args=server_args)
#    jupyter_comms=True)
visualizer.set_planar_viewpoint()

robot = station.get_controller_plant()
params = pydrake.manipulation.planner.DifferentialInverseKinematicsParameters(
    robot.num_positions(), robot.num_velocities())

time_step = 0.005
params.set_timestep(time_step)
iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                  iiwa14_velocity_limits))
# These constants are in body frame.
params.set_end_effector_velocity_gain([1, 0, 0, 0, 1, 1])
differential_ik = builder.AddSystem(
    pydrake.manipulation.planner.DifferentialInverseKinematicsIntegrator(
        robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
builder.Connect(differential_ik.get_output_port(),
                station.GetInputPort("iiwa_position"))

teleop = builder.AddSystem(PoseSliders(
    min_range = PoseSliders.MinRange(roll=0, x=-0.6, z=0.0),
    max_range = PoseSliders.MaxRange(roll=3.4, x=0.8, z=1.1),
    visible = PoseSliders.Visible(pitch=False, yaw=False, y=False)
))
builder.Connect(teleop.get_output_port(0), 
                differential_ik.get_input_port())
wsg_buttons = ToggleButtons(value=0.107, description="SchunkWsg", 
                            options=[('Open', 0.107), ('Close', 0.002)])
wsg_teleop = builder.AddSystem(WidgetSystem([wsg_buttons]))
builder.Connect(wsg_teleop.get_output_port(0),
                station.GetInputPort("wsg_position"))

diagram = builder.Build()
simulator = pydrake.systems.analysis.Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyMutableContextFromRoot(context)

q0 = station.GetOutputPort("iiwa_position_measured").Eval(
    station_context)
differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
differential_ik.SetPositions(diff_ik_context, q0)
teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

if running_as_notebook:  # Then we're not just running as a test on CI.
    simulator.set_target_realtime_rate(1.0)

    # Open the meshcat visualizer window (check your pop-up blocker).
    open_window(visualizer.vis.url(), "meshcat")

    stop_button = ToggleButton(value=False, description='Stop Simulation')
    display(stop_button)
    while not stop_button.value:
        simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
    stop_button.value = False

else:
    simulator.AdvanceTo(0.1)